<a href="https://colab.research.google.com/github/edmenciab733/ecc_qiskit_fallfest/blob/main/challenge_quantumquipu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
#!pip install qiskit
!pip install pylatexenc
#!pip install qiskit-aer
!pip install pennylane
!pip install ipython-autotime
!python -m pip install pennylane pennylane-lightning-gpu custatevec-cu11 --upgrade

In [2]:
%load_ext autotime

time: 314 µs (started: 2023-11-11 23:29:46 +00:00)


In [21]:

import pandas as pd
import pennylane  as qml
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MaxAbsScaler
import numpy as np


time: 583 µs (started: 2023-11-11 23:40:15 +00:00)


In [4]:
df_train = pd.read_csv("https://raw.githubusercontent.com/QuantumQuipu/QuantumChallenge/main/challenge_train.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/QuantumQuipu/QuantumChallenge/main/challenge_test.csv")

time: 413 ms (started: 2023-11-11 23:29:49 +00:00)


In [5]:
df_train.drop(df_train.columns[0], axis=1, inplace=True)
df_test.drop(df_test.columns[0], axis=1, inplace=True)

time: 15.6 ms (started: 2023-11-11 23:29:50 +00:00)


In [6]:
df_test.head()

,F1,F2,F3,F4,Target
0,1.866088,1.574254,0.138930,-0.351542,1
1,0.839427,-1.069518,-0.432655,-0.964800,0
2,0.557187,-0.921264,-0.379198,-0.003020,0
3,-0.694573,1.875495,0.533211,0.659735,1
4,-1.844339,-1.116098,1.074549,-0.156043,0


time: 14.8 ms (started: 2023-11-11 23:29:50 +00:00)


In [7]:
df_train.shape, df_test.shape

((800, 5), (200, 5))

time: 3.59 ms (started: 2023-11-11 23:29:50 +00:00)


In [8]:
df_train.describe()

x_train = df_train.iloc[:, :-1].values
y_train = df_train.iloc[:, -1].values
x_test = df_test.iloc[:, :-1].values
y_test = df_test.iloc[:, -1].values

time: 19.4 ms (started: 2023-11-11 23:29:50 +00:00)


In [9]:
scaler = MaxAbsScaler()
x_trscaler = scaler.fit_transform(x_train)

time: 1.64 ms (started: 2023-11-11 23:29:50 +00:00)


In [10]:
df_train["Target"].value_counts()

1    405
0    395
Name: Target, dtype: int64

time: 4.35 ms (started: 2023-11-11 23:29:50 +00:00)


In [11]:
x_trscaler

array([[-0.09351405,  0.51335636, -0.06345758,  0.52156551],
       [ 0.10503011,  0.43865928, -0.36516632,  0.14517601],
       [ 0.07311527,  0.41852907, -0.18166023, -0.22887792],
       ...,
       [ 0.21745571, -0.28535903, -0.33957967,  0.39072768],
       [-0.32939082,  0.14265007,  0.32825804, -0.47205864],
       [ 0.27475071,  0.45982156,  0.03640087, -0.21937293]])

time: 5.97 ms (started: 2023-11-11 23:29:50 +00:00)


In [19]:
x_tescaler = scaler.fit_transform(x_test)

time: 1.42 ms (started: 2023-11-11 23:38:57 +00:00)


In [23]:
np.min(x_trscaler), np.max(x_trscaler)

(-1.0, 1.0)

time: 3.56 ms (started: 2023-11-11 23:40:32 +00:00)


In [25]:
np.min(x_tescaler), np.max(x_tescaler)

(-1.0, 1.0)

time: 5.58 ms (started: 2023-11-11 23:41:08 +00:00)


# Encodig

## Amplitud Embedding

In [32]:
nqubits = 2
dev = qml.device("lightning.gpu", wires=nqubits)
@qml.qnode(dev)
def kernel_circuit_amplitude(a,b):
  normalize= True
  qml.AmplitudeEmbedding(a, wires=range(nqubits), normalize=normalize)
  qml.adjoint( qml.AmplitudeEmbedding(b, wires=range(nqubits), normalize=normalize) )
  return qml.probs(wires= range(nqubits))

time: 4.87 ms (started: 2023-11-11 23:55:56 +00:00)


In [16]:
drawer = qml.draw(kernel_circuit_amplitude)
print(drawer(x_trscaler[0], x_trscaler[3]))

0: ─╭|Ψ⟩─╭|Ψ⟩†─┤ ╭Probs
1: ─╰|Ψ⟩─╰|Ψ⟩†─┤ ╰Probs
time: 45.5 ms (started: 2023-11-11 23:30:13 +00:00)


In [17]:
kernel_circuit_amplitude(x_trscaler[0], x_trscaler[1]), kernel_circuit_amplitude(x_trscaler[0], x_trscaler[0])

(array([0.50326073, 0.38870318, 0.02799519, 0.0800409 ]),
 array([1.00000000e+00, 4.96867485e-32, 7.03479004e-33, 6.10562578e-32]))

time: 22 ms (started: 2023-11-11 23:30:15 +00:00)


## Angle Embedding

In [38]:
nqubits = 4
dev = qml.device("lightning.gpu", wires=nqubits)
@qml.qnode(dev)
def kernel_circuit_angle(a,b):
  normalize= True
  qml.AngleEmbedding(a, wires=range(nqubits))
  qml.adjoint( qml.AngleEmbedding(b, wires=range(nqubits)) )
  return qml.probs(wires= range(nqubits))

time: 6.67 ms (started: 2023-11-12 01:11:49 +00:00)


In [39]:
drawer = qml.draw(kernel_circuit_angle)
print(drawer(x_trscaler[0], x_trscaler[3]))

0: ─╭AngleEmbedding(M0)─╭AngleEmbedding(M1)†─┤ ╭Probs
1: ─├AngleEmbedding(M0)─├AngleEmbedding(M1)†─┤ ├Probs
2: ─├AngleEmbedding(M0)─├AngleEmbedding(M1)†─┤ ├Probs
3: ─╰AngleEmbedding(M0)─╰AngleEmbedding(M1)†─┤ ╰Probs
M0 = 
[-0.09351405  0.51335636 -0.06345758  0.52156551]
M1 = 
[-0.14544626  0.50351198 -0.24454744 -0.44697715]
time: 84.8 ms (started: 2023-11-12 01:12:14 +00:00)


## ZFeaturedMap

In [35]:
from itertools import combinations
def ZZFeatureMap(nqubits, data):
  nload = min(len(data), nqubits)
  for i in range(nload):
    qml.Hadamard(i)
    qml.RZ(2* data[i], wire = i)
  for pair in list(combinations(range(nload), 2 )):
    q0 = pair[0]
    q1 = pair[1]
    qml.CZ(wires=[q0, q1])
    qml.RZ(2* (np.pi -  data[q0]) * (np.pi -  data[q1]), wires = q1)
    qml.CZ(wires = [q0, q1])


nqubits = 4
dev = qml.device("lightning.gpu", wires=nqubits)
@qml.qnode(dev)
def kernel_circuit_zzfeature_map(a,b):
  normalize= True
  ZZFeatureMap(nqubits, a)
  qml.adjoint(ZZFeatureMap)(nqubits, b)
  return qml.probs(wires= range(nqubits))

time: 8.35 ms (started: 2023-11-12 01:10:03 +00:00)


# SVM

In [26]:
from sklearn import svm
svm  = svm.SVC().fit(x_trscaler, y_train)

time: 111 ms (started: 2023-11-11 23:41:46 +00:00)


In [27]:
y_predict = svm.predict(x_tescaler)
print(accuracy_score (y_predict, y_test ))

0.98
time: 7.26 ms (started: 2023-11-11 23:42:09 +00:00)


#QSVM

In [31]:
x_trscaler.shape

(800, 4)

time: 7.28 ms (started: 2023-11-11 23:55:11 +00:00)


In [33]:
from sklearn.svm import SVC

def qkernel(A, B):
  return np.array([[ kernel_circuit_amplitude(a, b) [0] for b in B ] for a in A])

svm = SVC(kernel = qkernel).fit(x_trscaler, y_train)

time: 57min 16s (started: 2023-11-11 23:56:08 +00:00)


In [34]:
y_predict = svm.predict(x_tescaler)
print(accuracy_score(y_predict, y_test ))

0.51
time: 13min 37s (started: 2023-11-12 00:54:28 +00:00)


In [41]:
def qkernel(A, B):
  return np.array([[ kernel_circuit_angle(a, b) [0] for b in B ] for a in A])

svm = SVC(kernel = qkernel).fit(x_trscaler, y_train)

time: 22min 1s (started: 2023-11-12 01:13:04 +00:00)


In [42]:
y_predict = svm.predict(x_tescaler)
print(accuracy_score(y_predict, y_test ))

0.975
time: 5min 32s (started: 2023-11-12 01:35:44 +00:00)
